<a href="https://colab.research.google.com/github/mkjubran/Fundamentals-of-AI-and-Machine-Learning/blob/main/PERFORMANCE_OF_MACHINE_LEARNING_SYSTEMS_%E2%80%93_K_FOLD_CROSS_VALIDATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PERFORMANCE OF MACHINE LEARNING SYSTEMS – K-FOLD CROSS-VALIDATION
